# Visualizing the performance of Choco-Q

Reproduce the Table II in the paper "[Choco-Q: Commute Hamiltonian-based QAOA for Constrained Binary Optimization][1]" (HPCA 2025).

[1]: https://ieeexplore.ieee.org/document/TBD

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)  # display all rows
pd.set_option('display.max_columns', None)  # display all columns.

## Reproduce Table II

In [2]:
problem_scale = 2
file_path = f"./scale_{problem_scale}"

df1 = pd.read_csv(f"{file_path}/evaluate_depth.csv")
grouped_df1 = df1.groupby(['pkid', 'layers', 'method'], as_index=False).agg({
    "culled_depth": 'mean',
})
pivot_df1 = grouped_df1.pivot(index=['pkid'], columns='method', values=["culled_depth"])
method_order1 = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
pivot_df1 = pivot_df1.reindex(columns=pd.MultiIndex.from_product([["culled_depth"], method_order1]))

df2 = pd.read_csv(f"{file_path}/evaluate_other.csv")
df2[['best_solution_probs', 'in_constraints_probs', 'iteration_count',
     'classcial', 'quantum', 'run_times']] = df2[['best_solution_probs', 'in_constraints_probs', 'iteration_count',
                                                  'classcial', 'quantum', 'run_times']].apply(pd.to_numeric, errors='coerce')
grouped_df2 = df2.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
    'iteration_count': 'mean',
    'classcial': 'mean',
    'run_times': 'mean',
})
pivot_df2 = grouped_df2.pivot(index=['pkid', 'variables', 'constraints'], columns='method', values=["best_solution_probs", 'in_constraints_probs', 'ARG'])
method_order2 = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
pivot_df2 = pivot_df2.reindex(columns=pd.MultiIndex.from_product([["best_solution_probs", 'in_constraints_probs', 'ARG'], method_order2]))

merged_df = pd.merge(pivot_df1, pivot_df2, on='pkid', how='inner')
merged_df = merged_df[['best_solution_probs', 'in_constraints_probs', 'ARG', 'culled_depth']]
merged_df = merged_df.rename(columns={
    'best_solution_probs': 'Success rate (%)',
    'in_constraints_probs': 'In-constraints rate (%)',
    'ARG': 'Approximation ratio gap (ARG)',
    'culled_depth': 'Circuit depth',
})
merged_df = merged_df.rename(columns={
    'PenaltySolver': 'Penalty',
    'CyclicSolver': 'Cyclic',
    'HeaSolver': 'HEA',
    'ChocoSolver': 'Choco-Q'
})
merged_df.index = [f"F{i}" for i in range(1, problem_scale + 1)] + [f"G{i}" for i in range(1, problem_scale + 1)] + [f"K{i}" for i in range(1, problem_scale + 1)]
merged_df

Success rate (%)                                 In-constraints rate (%)  \
            Penalty     Cyclic       HEA    Choco-Q                 Penalty   
F1        12.177734   7.197266  1.328125  82.402344               22.246094   
F2         0.058594   0.166016  0.000000  54.589844                0.917969   
G1         0.917969  11.884766  0.029297  56.337891                3.837891   
G2         0.019531   0.498047  0.000000   7.392578                0.419922   
K1         2.861328  18.027344  1.748047  83.066406                6.406250   
K2         0.000000   0.019531  0.000000  22.226562                0.019531   

                                 Approximation ratio gap (ARG)              \
       Cyclic        HEA Choco-Q                       Penalty      Cyclic   
F1  48.310547  10.859375   100.0                     33.194468   17.744957   
F2   4.072266   0.009766   100.0                     82.210030   57.944648   
G1  37.255859   0.341797   100.0                     96.402805   40.700281   
G2  20.986328   0.068359   100.0                    214.590636  106.759419   
K1  37.226562   3.984375   100.0                    213.352503   64.633532   
K2  60.117188   0.009766   100.0                    358.891573   66.409780   

                         Circuit depth                       
           HEA   Choco-Q       Penalty Cyclic   HEA Choco-Q  
F1   38.852053  0.071980          40.0   65.0  30.0    44.0  
F2  135.631438  0.347936          64.0   94.0  75.0   172.0  
G1  156.394436  0.235654         135.2  200.0  60.0   167.3  
G2  306.111009  0.867464         158.0  248.0  75.0   145.2  
K1  227.316007  0.168692          81.6  139.0  40.0   114.5  
K2  542.475797  2.014071         142.4  198.0  90.0   384.1

The results in this table may not completely align with those in Table II due to the random generation of benchmark configurations. Nevertheless, it is evident that Choco-Q demonstrates a significant advantage over other baselines. In the following, we calculate the improvement over Cyclic, the state-of-art baseline.

## Caculate the improvement over Cyclic

In [3]:
# Assuming 'merged_df' already contains the necessary data (after the previous steps)
# Calculate the improvement for each row

# Circuit depth improvement: cyclic / Choco-Q
merged_df['Circuit_depth_improvement'] = merged_df[('Circuit depth', 'Cyclic')] / merged_df[('Circuit depth', 'Choco-Q')]

# Success rate improvement: Choco-Q / cyclic
merged_df['Success_rate_improvement'] = merged_df[('Success rate (%)', 'Choco-Q')] / merged_df[('Success rate (%)', 'Cyclic')]

# In-constraints rate improvement: Choco-Q / cyclic
merged_df['In_constraints_rate_improvement'] = merged_df[('In-constraints rate (%)', 'Choco-Q')] / merged_df[('In-constraints rate (%)', 'Cyclic')]

# Filter out rows where any improvement column has a zero denominator or zero numerator (to avoid division by zero)
valid_rows = merged_df[(merged_df[('Circuit depth', 'Cyclic')] != 0) & (merged_df[('Circuit depth', 'Choco-Q')] != 0) &
                       (merged_df[('Success rate (%)', 'Cyclic')] != 0) & (merged_df[('Success rate (%)', 'Choco-Q')] != 0) &
                       (merged_df[('In-constraints rate (%)', 'Cyclic')] != 0) & (merged_df[('In-constraints rate (%)', 'Choco-Q')] != 0)]

# Calculate the average improvement for each metric
avg_circuit_depth_improvement = valid_rows['Circuit_depth_improvement'].mean()
avg_success_rate_improvement = valid_rows['Success_rate_improvement'].mean()
avg_in_constraints_rate_improvement = valid_rows['In_constraints_rate_improvement'].mean()

improvement_table = pd.DataFrame({
    'Circuit Depth': [avg_circuit_depth_improvement],
    'Success Rate': [avg_success_rate_improvement],
    'In-constraints Rate': [avg_in_constraints_rate_improvement]
}, index=['Improvement relative to Cyclic'])
improvement_table

,Circuit Depth,Success Rate,In-constraints Rate
Improvement relative to Cyclic,1.109449,250.410655,6.404186
